## Base de Dados
O objetivo final do projeto é usar uma base de dados com as relações de parentesco dentro do bairro de Surubeju do município de Monte Alegre - PA. No entanto, devido ao tempo de coleta desses dados, usarei um apenas uma amostra ou um conjunto de dados fake para a criação da visualização.

## Motivação
No referido bairro, no qual eu resido, existe uma peculiaridade de que os habitantes deste local tendem a constituir famílias com moradores do próprio bairro. Diante disso, criaram-se relações de parentesco peculiares, complexas e até mesmo desconhecidas pelos residentes. Com isso, decidi criar uma visualização dinâmica que demonstrasse todas as relações de parentesco presentes no bairro.

## Expectativa
O objetivo é conseguir plotar um gráfico de redes interativo, em que o usuário possa inserir o nome de duas pessoas e verificar a rede de parentesco entre elas.


## Esboços
Estão algumas amostras de dados com relações de parentesco e esboços de como será a visualização final.

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.colors as mcolors
import dash
import dash_html_components as html
import dash_cytoscape as cyto
import dash_core_components as dcc
from dash.dependencies import Input, Output
from networkx.algorithms.traversal.depth_first_search import dfs_tree
import numpy as np

C:\Users\LUIZLU~1\AppData\Local\Temp/ipykernel_20580/2034162194.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\LUIZLU~1\AppData\Local\Temp/ipykernel_20580/2034162194.py:7: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Coletando Data Frame do xls
full_data = pd.read_excel('familia.xlsx', index_col=0)
name_by_id = full_data[["nome"]].to_dict()["nome"]

In [4]:
# Criando o Grafo
family_tree = nx.DiGraph()
# Adicionando os nodes
for idx in full_data.index:
    short_name = name_by_id[idx].split(" ")[0]+" "+name_by_id[idx].split(" ")[-1]
    family_tree.add_node(str(idx),
                         id=str(idx),
                         short_name=short_name,
                         full_name=name_by_id[idx],
                        #  color=colors_by_id[idx]
                         )


In [5]:
# Adicionando Edges
father_ids = full_data["pai_id"].values
mother_ids = full_data["mae_id"].values
name_ids = full_data["nome"].index.values
edges_list = []

for name_id, father_id, mother_id in zip(name_ids, father_ids, mother_ids):
    if pd.notna(father_id):
        edges_list.append((str(name_id), str(int(father_id))))
    if pd.notna(mother_id):
        edges_list.append((str(name_id), str(int(mother_id))))


family_tree.add_edges_from(edges_list)


In [6]:
cyto_family_nodes = nx.readwrite.json_graph.cytoscape_data(family_tree)[
    'elements']['nodes']
cyto_family_edges = nx.readwrite.json_graph.cytoscape_data(family_tree)[
    'elements']['edges']
cyto_family_nodes[0]


{'data': {'id': '1',
  'short_name': 'Waldomiro Vasconcelos',
  'full_name': 'Waldomiro Vasconcelos',
  'value': '1',
  'name': '1'}}

In [7]:
family_degrees = family_tree.in_degree
max_degre = max([degree for id, degree in family_degrees])
green = np.array((57, 173, 51))
brown = np.array((51, 31, 15))
step = (brown-green)/max_degre

for node in cyto_family_nodes:
    degree = family_degrees[node["data"]["id"]]
    node["data"]["size"] = degree**2
    node["data"]["color"] =f"rgb{str(tuple(green+step*degree))}"


In [8]:
def get_path(family_tree:nx.classes.digraph.DiGraph, source_id:str, target_id:str):
    source = nx.descendants(family_tree,source_id)
    source.add(source_id)
    target = nx.descendants(family_tree,target_id)    
    target.add(target_id)
    intersec_parent_nodes = target.intersection(source)
    shortest_path_length = float("inf")
    all_shortest_paths = []
    for parent in intersec_parent_nodes:
        path_length =nx.shortest_path_length(family_tree,source_id,parent)+nx.shortest_path_length(family_tree,target_id,parent)
        if path_length < shortest_path_length:
           all_shortest_paths.clear()

        if path_length <= shortest_path_length:
           source_to_parent = [path for paths in nx.all_shortest_paths(family_tree,source_id,parent) for path in paths]
           target_to_parent = [path for paths in nx.all_shortest_paths(family_tree,target_id,parent) for path in reversed(paths[:-1])]
           all_shortest_paths.append(source_to_parent+target_to_parent)
           shortest_path_length = path_length

    return all_shortest_paths

[['70', '67', '2', '3', '7', '8', '9'], ['70', '67', '1', '3', '7', '8', '9']]

In [9]:
def dict_to_highlight_path(paths:list):
    edge_style = []
    node_style = []
    for node_path in paths:
            for idx in node_path[1:-1]:
                node_style +=[{"selector": f"#{idx}",
                            "style": {"background-color": "blue"}}]
            
            node_style +=[{"selector": f"#{node_path[0]}",
                            "style": {"background-color": "red"}}]
            node_style +=[{"selector": f"#{node_path[-1]}",
                            "style": {"background-color": "red"}}]

            for num, idx in enumerate(node_path[:-1]):
                edge_style += [
                    {'selector': f"[target = '{node_path[num]}' ][source = '{node_path[num+1]}']",
                    'style': {
                        'line-color': "blue",
                        'z-index':"10", 
                        "width":"10px", 
                        "opacity":"0.7"
                    }},
                    {'selector': f"[source = '{node_path[num]}' ][target = '{node_path[num+1]}']",
                    'style': {
                        'line-color': "blue",
                        'z-index':"10", 
                        "width":"10px", 
                        "opacity":"0.7"
                    }},
                ]

    return node_style + edge_style

In [10]:
cyto.load_extra_layouts()

In [11]:
cytoscape_stylesheet = [
    {'selector': 'node',
     'style': {'label': 'data(short_name)',
               'padding': 'data(size)',
               'border-width': '1px',
               'border-style': 'solid',
               'border-color': "black",
               "background-color":"data(color)",
               "display":"block",
               "word-wrap":"break-word"
               },
    },
    {'selector': 'edge',
     'style': {
         'line-color': "rgb(51, 31, 15)",
         'opacity': '0.5',
         "width":"5px"
     }},
]

app = dash.Dash("Family Network")
app.layout = html.Div([
    html.P('Dash Cytoscape:'),
    html.H2("Conheça a conexão entre duas pessoas do bairro"),
    html.Div([
        dcc.Dropdown(
            id='input-father-id',
            options=[{'label': name, 'value': idx}
                     for (idx, name) in name_by_id.items()],
            searchable=True,
            style={"margin": "5px", "width":"40vw"}
        ),
        dcc.Dropdown(
            id='input-mother-id',
            options=[{'label': name, 'value': idx}
                     for (idx, name) in name_by_id.items()],
            searchable=True,
            style={"margin": "5px", "width":"40vw"}
        ),
        ], style={"display":"flex",
                    "flex-direction":"row",
                    # "justify-content": "space-around",
                    }
        ),
    html.P( id="warning", style={"color":"red"}),
    html.Div([
        dcc.Dropdown(id='input-all-family',
            options=[{'label': name, 'value': idx}
                     for (idx, name) in name_by_id.items()],
            searchable=True,
            style={"margin": "5px", "width":"40vw"})
    ]),
    cyto.Cytoscape(
        id='cytoscape',
        elements=cyto_family_nodes+cyto_family_edges,
        layout={'name': 'dagre',
                #  'roots':'#1, #2'
                },
        style={'width': '100vw', 'height': '90vh'},
        stylesheet=cytoscape_stylesheet)
])


@app.callback(Output('cytoscape', 'elements'),
              Input('input-all-family', 'value'))
def get_all_ancestors(source_id):
    if source_id is not None:
        sub_tree = dfs_tree(family_tree, str(source_id))
        subtree_elements_nodes = nx.readwrite.json_graph.cytoscape_data(sub_tree)['elements']['nodes']
        subtree_elements_edges = nx.readwrite.json_graph.cytoscape_data(sub_tree)['elements']['edges']
        return subtree_elements_edges + subtree_elements_nodes     
   
    return cyto_family_edges+cyto_family_nodes


@app.callback(Output('cytoscape', 'stylesheet'),
              Output("warning", "children"),
              Input('input-father-id', 'value'),
              Input('input-mother-id', 'value'))
def highlight_node_path(source_id, target_id):
    style = []
    if source_id is not None and target_id is not None:
        path = get_path(family_tree,str(source_id),str(target_id))
        if len(path) == 0:
            return cytoscape_stylesheet+style, "Não há relação entre eles" 
        style = dict_to_highlight_path(path)       
    return cytoscape_stylesheet + style, ""
    


app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'Family Network' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Nov/2021 15:27:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 15:27:59] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 15:27:59] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 15:27:59] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 15:27:59] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 15:27:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 15:27:59] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
# Informações importantes
# Usar seletores de metadados como degree, indegree ou outdegree


## Ideias para a visualização

- Destacar todos os parents de um node
- 

<img src='Grafo.png' width="400" height="400">